In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
df_routes=pd.read_csv('../Files/Routes.csv')




In [ ]:
df_routes.head(2)

In [ ]:
df_incidents=pd.read_csv('../Files/Incidents.csv')

In [ ]:
df_routes.head(2)
df_incidents.head(2)

In [ ]:
# Convert 'date' column to datetime with the correct format
df_incidents['Incident_Date'] = pd.to_datetime(df_incidents['Incident_Date'])

In [ ]:
# Current date
current_date = datetime.now()

# Function to get the date 5 months ago
def date_n_months_ago(current_date, n):
    return current_date - timedelta(days=n*30)  # Approximate 5 months as 150 days

five_months_ago = date_n_months_ago(current_date, 5)

In [ ]:
# Adding columns to routes dataframe
df_routes['Total_Incidents_Count'] = 0
df_routes['high_incidents'] = 0
df_routes['high_last5Month_incidents'] = 0
df_routes['medium_incidents'] = 0
df_routes['medium_last5Month_incidents'] = 0
df_routes['low_incidents'] = 0
df_routes['low_last5Month_incidents'] = 0


In [ ]:


for index,row in df_routes.iterrows():
    #total incidents and count
    route_incidents=df_incidents[df_incidents['Route_Id']==row['Route_Id']]   
    incidents_count=route_incidents.shape[0]
    df_routes.loc[index, 'Total_Incidents_Count']=incidents_count
    
    #High incidents and count
    high_incidents = route_incidents[route_incidents['Severity'] == 'High']
    high_count=high_incidents.shape[0]
    high_last5Month_count=high_incidents[high_incidents['Incident_Date']>five_months_ago].shape[0]

    df_routes.loc[index, 'high_incidents']=(high_count-high_last5Month_count)
    df_routes.loc[index, 'high_last5Month_incidents']=high_last5Month_count

    
    
    #Medium incidents and count
    medium_incidents = route_incidents[route_incidents['Severity'] == 'Medium']
    medium_count=medium_incidents.shape[0]
    medium_last5Month_count=medium_incidents[medium_incidents['Incident_Date']>five_months_ago].shape[0]
    df_routes.loc[index, 'medium_incidents'] = (medium_count-medium_last5Month_count)
    df_routes.loc[index, 'medium_last5Month_incidents'] = medium_last5Month_count
    
        #Medium incidents and count
    low_incidents = route_incidents[route_incidents['Severity'] == 'Low']
    low_count=low_incidents.shape[0]
    low_last5Month_count=low_incidents[low_incidents['Incident_Date']>five_months_ago].shape[0]
    df_routes.loc[index, 'low_incidents'] = (low_count-low_last5Month_count)
    df_routes.loc[index, 'low_last5Month_incidents'] =low_last5Month_count
    


    
    


In [ ]:
df_routes.head(2)

In [ ]:
# Adding columns to routes dataframe
df_routes['Risk_Score'] = 0

In [ ]:

for index,row in df_routes.iterrows():
    total_travel_count=row['No_Of_Travels']
    Total_Incidents_Count=row['Total_Incidents_Count'] #200
    high_incidents=row['high_incidents'] #*3
    high_last5Month_incidents=row['high_last5Month_incidents'] #*5
    
    medium_incidents=row['medium_incidents'] #*2
    medium_last5Month_incidents=row['medium_last5Month_incidents'] #*3
    
    low_incidents=row['low_incidents'] #*1
    low_last5Month_incidents=row['low_last5Month_incidents'] #*2
    
    risk_value=((high_incidents*5)+(high_last5Month_incidents*7)+\
        (medium_incidents*3)+(medium_last5Month_incidents*5)+\
            (low_incidents*2)+(low_last5Month_incidents*3))/total_travel_count
        
    risk_value=risk_value*100;
    
    if risk_value>100:
        risk_value=98
    
    df_routes.loc[index,'Risk_Score']=risk_value
    
    

In [ ]:
df_routes.head()

In [ ]:
# Columns to drop
columns_to_drop = ['Route_Id',	'Origin_Port',	'Destination_Port'	,'Orgin_Port_Country','Travel_Direction'
                    ,'Destination_Port_Country','No_Of_Stops','Stops_Names',	'Stop_Points','Vessel_Type'		]  

# Drop columns (returns a new DataFrame)
new_df_routes = df_routes.drop(columns=columns_to_drop)
new_df_routes.head()

In [ ]:
# Split data into features and target variable
X = new_df_routes.drop(columns=['Risk_Score'])
y = new_df_routes['Risk_Score']

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [44]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [52]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Display the first few predictions
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(predictions_df.head())

Mean Squared Error: 110.60879344728613
        Actual  Predicted
158  25.188324  29.970629
500  25.056129  32.008984
396  23.433686  27.704127
155  18.858561  18.088142
321  80.986938  60.940848


In [55]:
new_df_routes.head(160)

,Distance,Average_Transit_Days,No_Of_Travels,Total_Incidents_Count,Risk_Score,high_incidents,high_last5Month_incidents,medium_incidents,medium_last5Month_incidents,low_incidents,low_last5Month_incidents
0,600,18,3161,136,15.881050,41,9,39,5,34,8
1,2468,29,2065,150,27.990315,47,12,39,11,36,5
2,3457,39,2908,141,16.403026,33,3,38,9,42,16
3,4500,15,2535,149,22.090730,49,8,35,11,39,7
4,200,27,1188,136,39.646465,29,7,33,11,45,11
...,...,...,...,...,...,...,...,...,...,...,...
155,385,72,2821,143,18.858561,43,9,33,9,37,12
156,429,71,561,143,91.622103,38,9,39,8,43,6
157,473,79,3215,143,16.080871,37,11,34,7,44,10
158,517,81,2124,152,25.188324,43,7,47,5,45,5


In [56]:
# Create a DataFrame with new data points to predict
new_data = pd.DataFrame({
    'Distance': [500, 850, 200],
    'Average_Transit_Days': [13, 25, 12],
    'No_Of_Travels': [1000, 2008, 3330],
    'Total_Incidents_Count': [600, 150, 300],
    'high_incidents': [400, 10, 45],
    'high_last5Month_incidents': [50, 0, 5],
    'medium_incidents': [50, 100, 100],
    'medium_last5Month_incidents': [0, 2, 50],
    'low_incidents': [50, 28, 100],
    'low_last5Month_incidents': [50, 10, 0]
})

In [57]:
predicted_scores = model.predict(new_data)

In [58]:
predicted_scores

array([190.41997346,  21.40792503,  41.23416896])

In [59]:
# Create a Pandas Series to store the predicted scores
predicted_scores_series = pd.Series(predicted_scores, index=new_data.index, name='Predicted_Risk_Score')

print(predicted_scores_series)

0    190.419973
1     21.407925
2     41.234169
Name: Predicted_Risk_Score, dtype: float64


In [60]:
import pickle

In [61]:
# Serialize the model to a file
with open('risk_model.pkl', 'wb') as f:
    pickle.dump(model, f)